# Horn Antenna Design for Hydrogen Line (1.42 GHz)

This notebook provides a starting point for designing a horn antenna tuned for the hydrogen line frequency at 1.42 GHz using OpenEMS.

## About the Project

The hydrogen line (21 cm line) is at 1420.405751 MHz and is used in radio astronomy to detect neutral hydrogen in space.

## Prerequisites

Before running this notebook, ensure you have:
1. OpenEMS installed on your system (see https://docs.openems.de/install.html)
2. Python dependencies installed: `pip install -r requirements.txt`

## References

- OpenEMS Documentation: https://docs.openems.de/
- OpenEMS Python Examples: https://github.com/thliebig/openEMS-Project

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import os
import sys

# Set up matplotlib for inline plotting
%matplotlib inline

print("Basic imports successful!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

In [ ]:
# Try to import OpenEMS
# Note: This will fail if OpenEMS is not installed
try:
    from CSXCAD import CSXCAD
    from openEMS import openEMS
    from openEMS.physical_constants import C0, EPS0, MUE0
    print("✓ OpenEMS imported successfully!")
    openems_available = True
except ImportError as e:
    print("✗ OpenEMS not found. Please install OpenEMS first.")
    print(f"  Error: {e}")
    print("\nInstallation instructions:")
    print("  Visit: https://docs.openems.de/install.html")
    openems_available = False

## Design Parameters

Let's define the basic parameters for our horn antenna design.

In [ ]:
# Hydrogen line frequency (21 cm line)
f0 = 1420.405751e6  # 1420.405751 MHz in Hz (precise hydrogen line frequency)
wavelength = 3e8 / f0  # Wavelength in meters

print(f"Design frequency: {f0/1e9:.6f} GHz ({f0/1e6:.6f} MHz)")
print(f"Wavelength (λ): {wavelength*1000:.2f} mm ({wavelength*100:.2f} cm)")

# Basic horn antenna parameters
# These are starting values that can be optimized
waveguide_width = wavelength / 2 * 0.9   # Slightly less than λ/2 for cutoff
waveguide_height = waveguide_width / 2    # Typical WR ratio
horn_length = 3 * wavelength              # Length of the horn
aperture_width = 2 * wavelength           # Output aperture width
aperture_height = 1.5 * wavelength        # Output aperture height

print(f"\nInitial Horn Dimensions:")
print(f"  Waveguide: {waveguide_width*1000:.1f} x {waveguide_height*1000:.1f} mm")
print(f"  Aperture: {aperture_width*1000:.1f} x {aperture_height*1000:.1f} mm")
print(f"  Length: {horn_length*1000:.1f} mm ({horn_length/wavelength:.1f}λ)")

## Horn Antenna Geometry Visualization

Let's visualize the basic horn antenna geometry before simulation.

In [ ]:
# Create a simple 2D visualization of the horn antenna (side view)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Side view (E-plane)
ax1.set_title('Horn Antenna - Side View (E-plane)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Length (mm)')
ax1.set_ylabel('Height (mm)')

# Draw horn outline
x = [0, 0, horn_length, horn_length, 0]
y_upper = [waveguide_height/2, waveguide_height/2, aperture_height/2, aperture_height/2, waveguide_height/2]
y_lower = [-waveguide_height/2, -waveguide_height/2, -aperture_height/2, -aperture_height/2, -waveguide_height/2]

ax1.plot([0, horn_length], [waveguide_height/2 * 1e3, aperture_height/2 * 1e3], 'b-', linewidth=2, label='Upper wall')
ax1.plot([0, horn_length], [-waveguide_height/2 * 1e3, -aperture_height/2 * 1e3], 'b-', linewidth=2, label='Lower wall')
ax1.plot([0, 0], [-waveguide_height/2 * 1e3, waveguide_height/2 * 1e3], 'b-', linewidth=2, label='Feed')
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax1.grid(True, alpha=0.3)
ax1.axis('equal')
ax1.set_xlim(-50, horn_length*1e3 + 50)

# Top view (H-plane)
ax2.set_title('Horn Antenna - Top View (H-plane)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Length (mm)')
ax2.set_ylabel('Width (mm)')

ax2.plot([0, horn_length], [waveguide_width/2 * 1e3, aperture_width/2 * 1e3], 'r-', linewidth=2, label='Right wall')
ax2.plot([0, horn_length], [-waveguide_width/2 * 1e3, -aperture_width/2 * 1e3], 'r-', linewidth=2, label='Left wall')
ax2.plot([0, 0], [-waveguide_width/2 * 1e3, waveguide_width/2 * 1e3], 'r-', linewidth=2, label='Feed')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.grid(True, alpha=0.3)
ax2.axis('equal')
ax2.set_xlim(-50, horn_length*1e3 + 50)

plt.tight_layout()
plt.show()

print(f"Flare angles:")
flare_angle_E = np.arctan((aperture_height - waveguide_height) / (2 * horn_length)) * 180 / np.pi
flare_angle_H = np.arctan((aperture_width - waveguide_width) / (2 * horn_length)) * 180 / np.pi
print(f"  E-plane: {flare_angle_E:.2f}°")
print(f"  H-plane: {flare_angle_H:.2f}°")

## OpenEMS Simulation Setup

This section will set up an OpenEMS simulation for the horn antenna.

**Note:** This requires OpenEMS to be properly installed on your system.

In [ ]:
if openems_available:
    print("Setting up OpenEMS simulation...")
    
    # Create simulation directory
    sim_path = os.path.join(os.getcwd(), 'horn_antenna_sim')
    os.makedirs(sim_path, exist_ok=True)
    print(f"Simulation directory: {sim_path}")
    
    # Unit conversion to mm (OpenEMS often works in mm)
    unit = 1e-3  # Coordinates in mm
    
    # Initialize FDTD and CSX
    FDTD = openEMS(NrTS=50000, EndCriteria=1e-4)
    FDTD.SetGaussExcite(f0, f0/10)  # Center frequency and bandwidth
    FDTD.SetBoundaryCond(['PML_8', 'PML_8', 'PML_8', 'PML_8', 'PML_8', 'PML_8'])  # PML boundary
    
    CSX = CSXCAD.ContinuousStructure()
    FDTD.SetCSX(CSX)
    
    print("✓ OpenEMS initialized")
    print("\nSimulation Parameters:")
    print(f"  Center frequency: {f0/1e9:.3f} GHz")
    print(f"  Max timesteps: 50000")
    print(f"  End criteria: 1e-4")
    print(f"  Boundary: 8-layer PML")
else:
    print("⚠ OpenEMS not available. Skipping simulation setup.")
    print("Install OpenEMS to run electromagnetic simulations.")

## Mesh Configuration

A proper mesh is crucial for accurate simulations. The mesh resolution should be fine enough to capture the geometry and field variations.

In [ ]:
if openems_available:
    # Mesh resolution
    mesh_res = wavelength / 20  # λ/20 is a good starting point
    
    print(f"Mesh resolution: {mesh_res*1000:.2f} mm (λ/{wavelength/mesh_res:.0f})")
    
    # Define mesh
    mesh = CSX.GetGrid()
    mesh.SetDeltaUnit(unit)
    
    # Simulation domain (with air space around antenna)
    air_space = 2 * wavelength  # Air space around antenna
    
    # Create mesh lines
    mesh.AddLine('x', np.arange(-air_space, horn_length + air_space, mesh_res) * 1000)  # to mm
    mesh.AddLine('y', np.arange(-air_space, air_space, mesh_res) * 1000)
    mesh.AddLine('z', np.arange(-air_space, air_space, mesh_res) * 1000)
    
    print("✓ Mesh configured")
    print(f"  Domain size: x=[{-air_space*1000:.0f}, {(horn_length+air_space)*1000:.0f}] mm")
    print(f"              y,z=[{-air_space*1000:.0f}, {air_space*1000:.0f}] mm")
else:
    print("⚠ OpenEMS not available. Skipping mesh configuration.")

## Material Definition

Define materials for the antenna structure (typically PEC - Perfect Electric Conductor for metal walls).

In [ ]:
if openems_available:
    # Create PEC material for metal walls
    metal = CSX.AddMetal('horn_metal')
    print("✓ PEC material created for horn walls")
    
    # Note: In a full implementation, you would now add the horn geometry
    # using metal.AddBox() or metal.AddPolygon() commands
    # This would include:
    # - Four tapered walls forming the horn
    # - Waveguide feed section
    
    print("\n💡 Next steps for full implementation:")
    print("  1. Add horn geometry using metal.AddBox() or parametric surfaces")
    print("  2. Define excitation port at waveguide input")
    print("  3. Add field probes and near-field to far-field transforms")
    print("  4. Run simulation with FDTD.Run()")
    print("  5. Post-process results (S-parameters, radiation pattern, gain)")
else:
    print("⚠ OpenEMS not available. Skipping material definition.")

## Expected Performance

Let's calculate some theoretical performance metrics for this horn antenna design.

In [ ]:
# Calculate theoretical gain (Kraus approximation for pyramidal horn)
aperture_area = aperture_width * aperture_height
aperture_efficiency = 0.51  # Typical for pyramidal horn
gain_linear = (4 * np.pi * aperture_area * aperture_efficiency) / (wavelength**2)
gain_dB = 10 * np.log10(gain_linear)

print("Theoretical Performance Estimates:")
print(f"  Aperture area: {aperture_area*1e6:.2f} cm²")
print(f"  Estimated gain: {gain_dB:.1f} dBi")
print(f"  Aperture efficiency: {aperture_efficiency*100:.0f}%")

# Beamwidth estimation
beamwidth_E = 70 * wavelength / aperture_height  # degrees
beamwidth_H = 70 * wavelength / aperture_width   # degrees

print(f"\nEstimated 3dB Beamwidth:")
print(f"  E-plane: {beamwidth_E:.1f}°")
print(f"  H-plane: {beamwidth_H:.1f}°")

# Directivity
print(f"\nDirectivity: {gain_dB:.1f} dB")

# Effective aperture
Ae = (wavelength**2 * gain_linear) / (4 * np.pi)
print(f"Effective aperture: {Ae*1e4:.2f} cm²")

## Next Steps

To complete the horn antenna design:

1. **Geometry Implementation**: Add the complete horn geometry to the CSX structure
2. **Excitation**: Define the waveguide port excitation
3. **Field Probes**: Add E-field and H-field probes
4. **NF2FF**: Set up near-field to far-field transformation for radiation patterns
5. **Run Simulation**: Execute the FDTD simulation
6. **Post-Processing**: 
   - S-parameters (return loss, VSWR)
   - Radiation patterns (E-plane and H-plane)
   - Gain and directivity
   - Impedance matching
7. **Optimization**: Adjust dimensions for optimal performance at 1.42 GHz

### Design Considerations for 1.42 GHz Hydrogen Line:

- **Bandwidth**: Consider the required bandwidth for Doppler shift detection
- **Side lobes**: Low side lobes are important for radio astronomy
- **Polarization**: Linear polarization is typically used
- **Feed network**: May need to design a waveguide-to-coax transition
- **LNA integration**: Plan for low-noise amplifier at the feed point

## Resources and References

### OpenEMS Resources
- [OpenEMS Documentation](https://docs.openems.de/)
- [OpenEMS Python Examples](https://github.com/thliebig/openEMS-Project/tree/master/python)
- [OpenEMS Tutorial](https://openems.de/index.php/Tutorial)

### Horn Antenna Design
- Balanis, C.A. "Antenna Theory: Analysis and Design"
- Kraus, J.D. "Antennas for All Applications"
- Love, A.W. "Electromagnetic Horn Antennas"

### Hydrogen Line Radio Astronomy
- [SARA - Society of Amateur Radio Astronomers](https://www.radio-astronomy.org/)
- [Radio-Sky Publishing](http://www.radio-sky.com/)
- [NRAO Essentials of Radio Astronomy](https://science.nrao.edu/opportunities/courses/era/)

In [ ]:
# Summary
print("="*70)
print(" HORN ANTENNA DESIGN SUMMARY")
print("="*70)
print(f"\nDesign Frequency: {f0/1e6:.2f} MHz (Hydrogen Line)")
print(f"Wavelength: {wavelength*100:.2f} cm")
print(f"\nWaveguide Dimensions: {waveguide_width*1000:.1f} × {waveguide_height*1000:.1f} mm")
print(f"Aperture Dimensions: {aperture_width*1000:.1f} × {aperture_height*1000:.1f} mm")
print(f"Horn Length: {horn_length*1000:.1f} mm")
print(f"\nEstimated Gain: {gain_dB:.1f} dBi")
print(f"Beamwidth: {beamwidth_E:.1f}° (E) × {beamwidth_H:.1f}° (H)")
print("\nStatus:")
if openems_available:
    print("  ✓ OpenEMS is ready for simulation")
else:
    print("  ✗ OpenEMS needs to be installed")
print("="*70)